<a href="https://colab.research.google.com/github/ilkersigirci/ML-with-Colab/blob/master/quora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pprint
import tensorflow as tf
tf.reset_default_graph()
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from sklearn.metrics import log_loss
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#embeddings imports
import gensim
from gensim.models import Word2Vec
from gensim import corpora
import gensim.downloader as api
from gensim.matutils import softcossim
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
from tqdm import tqdm_notebook
from nltk import word_tokenize
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#!pip install -U -q kaggle
#!mkdir -p ~/.kaggle
#!cp drive/My\ Drive/Colab\ Notebooks/kaggle.json  ~/.kaggle/
#!kaggle datasets list
#!kaggle competitions download -c quora-question-pairs -p "/content/mydata/quora"

In [0]:
!mkdir mydata
!mkdir mydata/quora

In [0]:
!cp  drive/My\ Drive/Colab\ Notebooks/Dataset/quora/{train.csv,test.csv} mydata/quora/

In [0]:
df = pd.read_csv('./mydata/quora/train.csv')
df.dropna(axis = 0, inplace = True)

In [0]:
df.groupby("is_duplicate")['id'].count().plot(kind='bar')

In [0]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)

In [0]:
df.head()

In [0]:
#cleaning text
import re
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}

def pad_str(s):
    return ' '+s+' '

def pad_number(pattern):
    matched_string = pattern.group(0)
    return pad_str(matched_string)

def pad_pattern(pattern):
    matched_string = pattern.group(0)
    return pad_str(matched_string)
    
def clean(text, stem_words=True):
    
    if pd.isnull(text):
        return ''
    
    # Empty question   
    if type(text) != str or text=='':
        return ''
    
    # Clean the text, with the option to stem words.
    # stops = set(stopwords.words("english"))       

    # Clean the text
    text = re.sub("\'s", " ",                       text, flags=re.IGNORECASE)
    text = re.sub(" whats ", " what is ",           text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ",                 text, flags=re.IGNORECASE)
    text = re.sub("can't", "can not",               text, flags=re.IGNORECASE)
    text = re.sub("n't", " not ",                   text, flags=re.IGNORECASE)
    text = re.sub("i'm", "i am",                    text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ",                  text, flags=re.IGNORECASE)
    text = re.sub("\'d", " would ",                 text, flags=re.IGNORECASE)
    text = re.sub("\'ll", " will ",                 text, flags=re.IGNORECASE)
    text = re.sub("e\.g\.", " eg ",                 text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ",                 text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ",        text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ",                     text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ",         text, flags=re.IGNORECASE)
    text = re.sub("e-mail", " email ",              text, flags=re.IGNORECASE)
    text = re.sub('\$', " dollar ",                 text, flags=re.IGNORECASE)
    text = re.sub('\%', " percent ",                text, flags=re.IGNORECASE)
    text = re.sub('\&', " and ",                    text, flags=re.IGNORECASE)    
    text = re.sub("(?<=[0-9])rs ", " rs ",          text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ",           text, flags=re.IGNORECASE)
    text = re.sub(r" UK ", " England ",             text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ",            text, flags=re.IGNORECASE)
    text = re.sub(r" switzerland ", " Switzerland ",text, flags=re.IGNORECASE)
    text = re.sub(r" china ", " China ",            text, flags=re.IGNORECASE)
    text = re.sub(r" chinese ", " Chinese ",        text, flags=re.IGNORECASE) 
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ",     text, flags=re.IGNORECASE)
    text = re.sub(r" quora ", " Quora ",            text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ",    text, flags=re.IGNORECASE)
    text = re.sub(r" actived ", " active ",         text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ",         text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ",    text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote",           text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ",           text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ",                text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ",      text, flags=re.IGNORECASE)
    text = re.sub(r" android "," operating system ",text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ",   text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ",                text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ",                text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" dna ", " DNA ",                text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ",                  text, flags=re.IGNORECASE)
    text = re.sub(r" banglore ", " Banglore ",      text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ",                 text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ",              text, flags=re.IGNORECASE)

    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE)
    text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text)

    text =re.sub('(?<=[0-9])\,(?=[0-9])',"",text, flags=re.IGNORECASE) # remove comma between numbers
    
    # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ",         text, flags=re.IGNORECASE)

    #text = re.sub('[\!\?\@\^\+\*\/\,\~\|\`\=\:\;\.\#\\\]', pad_pattern, text)        
    #text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    #text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    
    # all numbers should separate from words, this is too aggressive
    # text = re.sub('[0-9]+', pad_number,  text, flags=re.IGNORECASE)  
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation]).lower()
        
    return text  

In [0]:
df['question1'] = df['question1'].apply(clean)
df['question2'] = df['question2'].apply(clean)

In [0]:
dfSub =  df

In [0]:
#BoW + XgBoost Model
# train -> 0.82 / test -> 0.75

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = count_vect.transform(df['question1'].values)
trainq2_trans = count_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)

In [0]:
from sklearn.metrics import f1_score, classification_report, accuracy_score

print('training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

training score: 0.8292012350767584
validation score: 0.7590104068777661
              precision    recall  f1-score   support

           0       0.79      0.90      0.84     84267
           1       0.78      0.60      0.68     49148

    accuracy                           0.79    133415
   macro avg       0.78      0.75      0.76    133415
weighted avg       0.79      0.79      0.78    133415



In [0]:
#Word level TF-IDF
# train -> 0.84 / test -> 0.75

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
#Combine it with xgboost
from sklearn.metrics import f1_score, classification_report, accuracy_score
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)
print('word level tf-idf training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('word level tf-idf validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

word level tf-idf training score: 0.8893228534163832
word level tf-idf validation score: 0.761087480236885
              precision    recall  f1-score   support

           0       0.79      0.90      0.84     84267
           1       0.78      0.60      0.68     49148

    accuracy                           0.79    133415
   macro avg       0.79      0.75      0.76    133415
weighted avg       0.79      0.79      0.78    133415



In [0]:
#N-gram(2,3) level TF-IDF
# train -> 0.71 / test -> 0.67

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect_ngram.transform(df['question1'].values)
trainq2_trans = tfidf_vect_ngram.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
#Combine it with xgboost
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)
print('n-gram level tf-idf training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('n-gram level tf-idf validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

In [0]:
#Character Level TF-IDF + XgBoost -> HIGHEST
# train -> 0.98 / test -> 0.80

from sklearn.metrics import f1_score, classification_report, accuracy_score
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect_ngram_chars.transform(df['question1'].values)
trainq2_trans = tfidf_vect_ngram_chars.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)
print('character level tf-idf training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('character level tf-idf validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

In [0]:
#######################################################################################################
##################################### Word Embeddings Test ############################################
#######################################################################################################

In [0]:
import gensim
from gensim.models import Word2Vec

In [0]:
df = pd.read_csv('./mydata/quora/train.csv')
df = df.dropna(how="any").reset_index(drop=True)

In [0]:
#Word Mover's Distance (WMD)

#Copy GoogleNews-vectors-negative300.bin.gz
!cp  drive/My\ Drive/Colab\ Notebooks/Dataset/quora/GoogleNews-vectors-negative300.bin.gz mydata/quora/

googlenews_corpus_path = 'mydata/quora/GoogleNews-vectors-negative300.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(googlenews_corpus_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
question1 = 'What would a Trump presidency mean for current international master’s students on an F1 visa?'
question2 = 'How will a Trump presidency affect the students presently in US or planning to study in US?'

question1 = question1.lower().split()
question2 = question2.lower().split()

question1 = [w for w in question1 if w not in stop_words]
question2 = [w for w in question2 if w not in stop_words]

#normalization decreases the length in duplicate labeled question
model.init_sims(replace=True) #normalize word2vec vectors
distance = model.wmdistance(question1, question2)             #distance = 0.7589
print('distance = %.4f' % distance)
################################################################################
question3 = 'Why am I mentally very lonely? How can I solve it?'
question4 = 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?'

question3 = question3.lower().split()
question4 = question4.lower().split()

question3 = [w for w in question3 if w not in stop_words]
question4 = [w for w in question4 if w not in stop_words]

#normalization doesn't affect the length in non duplicate labeled question
model.init_sims(replace=True) #normalize word2vec vectors
distance = model.wmdistance(question3, question4)             #distance = 1.2637
print('distance = %.4f' % distance)

In [0]:
#soft cosine
from gensim import corpora

documents = [question1, question2, question3, question4]
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

#Convert the sentences into bag-of-words vectors.
question1 = dictionary.doc2bow(question1)
question2 = dictionary.doc2bow(question2)
question3 = dictionary.doc2bow(question3)
question4 = dictionary.doc2bow(question4)

import gensim.downloader as api

w2v_model = api.load("glove-wiki-gigaword-50")
similarity_matrix = w2v_model.similarity_matrix(dictionary)

from gensim.matutils import softcossim

#similarity = 0.7611
similarity = softcossim(question1, question2, similarity_matrix)
print('similarity = %.4f' % similarity)

#similarity = 0.2030
similarity = softcossim(question3, question4, similarity_matrix)
print('similarity = %.4f' % similarity)

In [0]:
#FuzzyWuzzy
from fuzzywuzzy import fuzz

question1 = 'What would a Trump presidency mean for current international master’s students on an F1 visa?'
question2 = 'How will a Trump presidency affect the students presently in US or planning to study in US?'
question3 = 'Why am I mentally very lonely? How can I solve it?'
question4 = 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?'

# ratio            -> 51 -> compares the entire string similarity, in order
# partial_ratio    -> 52 -> compares partial string similarity.
# token_sort_ratio -> 46 -> ignores word order.
# token_set_ratio  -> 52 -> ignores duplicated words


fuzz.partial_token_set_ratio(question1, question2) # 100
fuzz.partial_token_set_ratio(question3, question4) # 37

100

In [0]:
#######################################################################################################
############################## Word Embeddings Feature Engineering ####################################
#######################################################################################################

In [0]:
def wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return model.wmdistance(q1, q2)

def norm_wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return norm_model.wmdistance(q1, q2)

def my_wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return model.wmdistance(q1, q2)
  

nltk.download('punkt')
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    print(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)

In [0]:
### NEW FEATURES ###

# The length of word.
# The length of character.
# The length of common word between question1 and question2.
# The length difference between question1 and question2.
# The Cosine distance between vectors question1 and question2.
# The City block (Manhattan) distance between vectors question1 and question2.
# The Jaccard distance between vectors question1 and question2.
# The Canberra distance between vectors question1 and question2.
# The Euclidean distance between vectors question1 and question2.
# The Minkowski distance between vectors question1 and question2.
# The Bray-Curtis distance between vectors question1 and question2.
# The skewness and kurtosis of vectors question1 and question2.
# WMD
# normalized WMD

In [0]:
df['len_q1'] = df.question1.apply(lambda x: len(str(x)))
print("len_q1 done..")
df['len_q2'] = df.question2.apply(lambda x: len(str(x)))
print("len_q2 done..")
df['diff_len'] = df.len_q1 - df.len_q2
print("diff_len done..")
df['len_char_q1'] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
print("len_char_q1 done..")
df['len_char_q2'] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
print("len_char_q2 done..")
df['len_word_q1'] = df.question1.apply(lambda x: len(str(x).split()))
print("len_word_q1 done..")
df['len_word_q2'] = df.question2.apply(lambda x: len(str(x).split()))
print("len_word_q2 done..")
df['common_words'] = df.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
print("common_words done..")
df['fuzz_ratio'] = df.apply(lambda x: fuzz.ratio(str(x['question1']), str(x['question2'])), axis=1)
print("fuzz_ratio done..")
df['fuzz_partial_ratio'] = df.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
print("fuzz_partial_ratio done..")
df['fuzz_partial_token_set_ratio'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
print("fuzz_partial_token_set_ratio done..")
df['fuzz_partial_token_sort_ratio'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
print("fuzz_partial_token_sort_ratio done..")
df['fuzz_token_set_ratio'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
print("fuzz_token_set_ratio done..")
df['fuzz_token_sort_ratio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
print("fuzz_token_sort_ratio done..")

len_q1 done..
len_q2 done..
diff_len done..
len_char_q1 done..
len_char_q2 done..
len_word_q1 done..
len_word_q2 done..
common_words done..
fuzz_ratio done..
fuzz_partial_ratio done..
fuzz_partial_token_set_ratio done..
fuzz_partial_token_sort_ratio done..
fuzz_token_set_ratio done..
fuzz_token_sort_ratio done..


In [0]:
df.head()

In [0]:
dfExtraFeatures = df

In [0]:
#word2vec modeling
#googlenews_corpus_path = 'mydata/quora/GoogleNews-vectors-negative300.bin.gz'
#model = gensim.models.KeyedVectors.load_word2vec_format(googlenews_corpus_path, binary=True)
#df['wmd'] = df.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)
df['wmd'] = df.apply(lambda x: my_wmd(x['question1'], x['question2']), axis=1)

#normalized word2vec modeling
#norm_model = gensim.models.KeyedVectors.load_word2vec_format(googlenews_corpus_path, binary=True)
#norm_model.init_sims(replace=True)
#df['norm_wmd'] = df.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)
model.init_sims(replace=True)
df['norm_wmd'] = df.apply(lambda x: my_wmd(x['question1'], x['question2']), axis=1)

In [0]:
df.to_csv("quora_middle.csv")

In [0]:
question1_vectors = np.zeros((df.shape[0], 300))

for i, q in enumerate(tqdm_notebook(df.question1.values)):
    question1_vectors[i, :] = sent2vec(q)
    
question2_vectors  = np.zeros((df.shape[0], 300))
for i, q in enumerate(tqdm_notebook(df.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

In [0]:
df['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
df['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
df['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
df['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

In [0]:
df['is_duplicate'].value_counts()

In [0]:
# cosine_distance, jaccard_distance and braycurtis_distance have null values
df.isnull().sum()

In [0]:
df.drop(['question1', 'question2'], axis=1, inplace=True)
df = df[pd.notnull(df['cosine_distance'])]
df = df[pd.notnull(df['jaccard_distance'])]

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

X = df.loc[:, df.columns != 'is_duplicate']
y = df.loc[:, df.columns == 'is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [0]:

import xgboost as xgb

model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train.values.ravel()) 
prediction = model.predict(X_test)
cm = confusion_matrix(y_test, prediction)  
print(cm)  
print('Accuracy', accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))